In [ ]:
import requests
print('Requests version: ' + requests.__version__)

import bs4
print('Beautiful Soup version: ' + bs4.__version__)
from bs4 import BeautifulSoup

In [ ]:
# https://proxiesapi.com/blog/scraping-listings-from-realtor-with-python-and-bea.html.php
headers = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.11 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9',
'Accept-Encoding': 'identity'
}
#'Accept-Encoding': 'identity'

url = 'https://www.realtor.ca/on/toronto/condos-for-sale'

response=requests.get(url,headers=headers)

#print(response.content)

soup=BeautifulSoup(response.content)

In [ ]:
sales = soup.find_all(attrs={'class':'blockLink listingDetailsLink'})
sales

In [43]:
sale = sales[0]
print(sale.prettify())

In [51]:
# extract url for each sale
link = sale.attrs['href']
link

'/real-estate/23826918/1002-1338-york-mills-rd-toronto-parkwoods-donalda'

### Extract sale info for one building

In [54]:
response1=requests.get('https://www.realtor.ca/' + link,headers=headers)

sale_soup=BeautifulSoup(response1.content)
sale_soup


<!DOCTYPE html>

<html lang="en" xmlns="http://www.w3.org/1999/xhtml">
<head><script async="" src="/dnight-Exit-shall-Braith-Then-why-vponst-is-proc"></script><meta charset="utf-8"/><meta content="#D0191D" name="theme-color"/><meta content="width=device-width" name="viewport"/><meta content="notranslate" name="google"/><meta content="NO-CACHE" http-equiv="CACHE-CONTROL"/><meta content="NO-CACHE" http-equiv="PRAGMA"/><meta content="NOARCHIVE" name="GOOGLEBOT"/><meta content="telephone=no" name="format-detection"/><meta content="a2430a87e639e6cf0f00273171d97103" name="shareaholic:site_id"/><link as="style" href="/bundles/css/desktop/page/listing" rel="prefetch"/>
<link as="style" href="/bundles/css/desktop/page/map/locallogic" rel="prefetch"/><link as="script" href="/bundles/js/desktop/page/listing" rel="prefetch"/>
<link as="script" href="/bundles/js/desktop/page/map" rel="prefetch"/>
<link as="script" href="/bundles/js/desktop/page/index" rel="prefetch"/>
<link href="https://www.realt

In [56]:
sale_property = sale_soup.find_all(attrs={'id':'GTMDataLayerCode'})
sale_property

[<script id="GTMDataLayerCode" type="text/javascript">
                          window.dataLayer = window.dataLayer || [];
                         dataLayer.push({ 
                            event: 'propertyDetailsLoaded',
                            pagePath: (location.pathname + (location.search || '')),
                            pageTitle: document.title,
                            visitor: {
                                loggedIn: 'No',
                                userId: '',
                                favouriteCountValue: '0',
                                saveSearchCountValue: '0'
                            },
                            site: {
                                language: 'EN'
                            },
                             property: {            
                                 propertyID: '23826918',
                                 listingID: 'C5432600',
                                 propertyType: 'Single Family',
           

In [85]:
property_lst = sale_property[0].string.split("property: {")[1].split('\r\n')[1:-3]
#property_lst

property_dict = {}
for i in range(len(property_lst)):
    property_lst[i] = property_lst[i].strip(", ")
    tmp = property_lst[i].split(": ")
    property_dict[tmp[0]] = tmp[1].strip("''")
    
    
property_dict

["                                propertyID: '23826918',",
 "                                listingID: 'C5432600',",
 "                                propertyType: 'Single Family',",
 "                                price: '485888.00',",
 "                                leasePrice: '',",
 "                                buildingType: 'Apartment',",
 "                                neighbourhood: '',                    ",
 "                                bathrooms: '2',",
 "                                bedrooms: '3',",
 "                                communityFeatures: 'Pets not Allowed',",
 "                                nearbyAmenities: 'Hospital, Park, Public Transit, Schools',",
 "                                storeys: '',",
 "                                walkscore: '',",
 "                                transitscore: '',",
 "                                machineryIncluded: '',",
 "                                rentalEquipment: '',",
 "                      

### get more details - building

In [52]:
# get address
sale_address = sale_soup.find(attrs={'class':'listingTopDetailsLeft'}).h1.contents # can also use .text
sale_address

['#520 -30 MEADOWGLEN PL', <br/>, 'Toronto, Ontario M1G0A6']

In [72]:
# building_info_dict
building_info_dict = dict()
for info in soup_building_info:
    building_info_dict[info.find(attrs={'class':'propertyDetailsSectionContentLabel'}).string] = info.find(attrs={'class':'propertyDetailsSectionContentValue'}).text    

In [73]:
building_info_dict

{'Property Type': 'Single Family',
 'Building Type': 'Apartment',
 'Community Name': 'Woburn',
 'Title': 'Condominium/Strata',
 'Annual Property Taxes': '$2,319.06 (CAD)',
 'Parking Type': 'Underground, Visitor Parking',
 'Time on REALTOR.ca': '2 hours',
 'Above Grade': '2',
 'Total': '2',
 'Features': 'Balcony',
 'Building Amenities': 'Storage - Locker, Security/Concierge, Party Room, Exercise Centre',
 'Cooling': 'Central air conditioning',
 'Heating Type': 'Forced air (Natural gas)',
 'Exterior Finish': 'Concrete',
 'Pool Type': 'Outdoor pool',
 'Community Features': 'Community Centre',
 'Amenities Nearby': 'Hospital, Park, Public Transit, Schools',
 'Maintenance Fees': '$502 (CAD) Monthly',
 'Maintenance Management Company': 'Rab Property Management',
 'Total Parking Spaces': '1'}

In [75]:
building_info_dict.keys()
# next(iter(my_dict))

dict_keys(['Property Type', 'Building Type', 'Community Name', 'Title', 'Annual Property Taxes', 'Parking Type', 'Time on REALTOR.ca', 'Above Grade', 'Total', 'Features', 'Building Amenities', 'Cooling', 'Heating Type', 'Exterior Finish', 'Pool Type', 'Community Features', 'Amenities Nearby', 'Maintenance Fees', 'Maintenance Management Company', 'Total Parking Spaces'])

### loop through each building to extract info

In [136]:
# key_lst = list(full_dict.keys())
# print(key_lst)
key_lst = ['Address1', 'Address2', 'propertyID', 'listingID', 'propertyType', 'price', 'leasePrice', 'buildingType', 'neighbourhood', 'bathrooms', 'bedrooms', 'communityFeatures', 'nearbyAmenities', 'storeys', 'walkscore', 'transitscore', 'machineryIncluded', 'rentalEquipment', 'parkingType', 'majorBusinessType', 'minorBusinessType', 'totalUnits', 'landSize', 'zoningType', 'interiorFloorSpace', 'exteriorBuildingSize', 'franchise', 'buildingAmenities', 'buildingStyle', 'basementDevelopment', 'notificationUpdate', 'anchor', 'retailStorefront', 'clearCeilingHeight', 'hasphoto', 'photos', 'multimediaFeatures', 'listingVideoType', 'city', 'province', 'soldData', 'hasAlternateFeatureSheet', 'hasOpenHouse', 'hasVirtualOpenHouse', 'hasCMHCFTHBBanner', 'hasTDpreApprovalIcon', 'hasActiveVirtualOpenHouse', 'Property Type', 'Building Type', 'Community Name', 'Title', 'Annual Property Taxes', 'Parking Type', 'Time on REALTOR.ca', 'Above Grade', 'Total', 'Features', 'Building Amenities', 'Cooling', 'Heating Type', 'Exterior Finish', 'Pool Type', 'Community Features', 'Amenities Nearby', 'Maintenance Fees', 'Maintenance Management Company', 'Total Parking Spaces']

['Address1', 'Address2', 'propertyID', 'listingID', 'propertyType', 'price', 'leasePrice', 'buildingType', 'neighbourhood', 'bathrooms', 'bedrooms', 'communityFeatures', 'nearbyAmenities', 'storeys', 'walkscore', 'transitscore', 'machineryIncluded', 'rentalEquipment', 'parkingType', 'majorBusinessType', 'minorBusinessType', 'totalUnits', 'landSize', 'zoningType', 'interiorFloorSpace', 'exteriorBuildingSize', 'franchise', 'buildingAmenities', 'buildingStyle', 'basementDevelopment', 'notificationUpdate', 'anchor', 'retailStorefront', 'clearCeilingHeight', 'hasphoto', 'photos', 'multimediaFeatures', 'listingVideoType', 'city', 'province', 'soldData', 'hasAlternateFeatureSheet', 'hasOpenHouse', 'hasVirtualOpenHouse', 'hasCMHCFTHBBanner', 'hasTDpreApprovalIcon', 'hasActiveVirtualOpenHouse', 'Property Type', 'Building Type', 'Community Name', 'Title', 'Annual Property Taxes', 'Parking Type', 'Time on REALTOR.ca', 'Above Grade', 'Total', 'Features', 'Building Amenities', 'Cooling', 'Heating T

In [149]:
sales_info = []

for i in range(len(sales)):
    print(i)
    sale = sales[i]
    link = sale.attrs['href']
    response=requests.get('https://www.realtor.ca/' + link,headers=headers2)
    sale_soup=BeautifulSoup(response.content)
    
    # extract from json property dictionary
    sale_property = sale_soup.find_all(attrs={'id':'GTMDataLayerCode'})
    property_lst = sale_property[0].string.split("property: {")[1].split('\r\n')[1:-3]
    property_dict = {}
    for i in range(len(property_lst)):
        property_lst[i] = property_lst[i].strip(", ")
        tmp = property_lst[i].split(": ")
        property_dict[tmp[0]] = tmp[1].strip("''")
    
    # extract more info
    sale_address = sale_soup.find(attrs={'class':'listingTopDetailsLeft'}).h1.contents
    address_dict = {'Address1': sale_address[0], 'Address2': sale_address[2]}
    soup_building_info = sale_soup.find_all(attrs={'class':'propertyDetailsSectionContentSubCon'})
    building_info_dict = dict()
    for info in soup_building_info:
        building_info_dict[info.find(attrs={'class':'propertyDetailsSectionContentLabel'}).string] = info.find(attrs={'class':'propertyDetailsSectionContentValue'}).text    
    
    full_dict = address_dict.copy() 
    full_dict.update(property_dict)
    full_dict.update(building_info_dict)
    
    # add missing field with ''
    for key in key_lst:
        if key not in list(full_dict.keys()):
            full_dict[key] = ''
            
    reordered_dict = {k: full_dict[k] for k in key_lst}
    
    sales_info.append(reordered_dict)

0
1
2
3
4


IndexError: list index out of range

In [153]:
### save sales info to csv
# https://www.geeksforgeeks.org/how-to-save-a-python-dictionary-to-a-csv-file/
import csv

with open('realtor.csv', 'w') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames = key_lst)
    writer.writeheader()
    writer.writerows(sales_info)